In [2]:
from app import app, db, Project, ProjectUser, User

with app.app_context():
    for project in Project.query.all():
        # Check if an owner association already exists
        has_owner = any(assoc.role == 'owner' for assoc in project.user_associations)
        if not has_owner:
            # Check again to avoid duplicate entries
            existing_assoc = ProjectUser.query.filter_by(
                user_id=project.user_id,
                project_id=project.id
            ).first()

            if not existing_assoc:
                user = db.session.get(User, project.user_id)  # SQLAlchemy 2.0 way
                if user:
                    assoc = ProjectUser(user=user, project=project, role='owner')
                    db.session.add(assoc)

    db.session.commit()
